In [ ]:
import time
import os
import tkinter as tk
from tkinter import scrolledtext
import winsound  # Para reproducir sonido en Windows (usar `afplay` en macOS o `aplay` en Linux)

def play_alert_sound():
    try:
        winsound.Beep(1000, 500)  # Sonido de alerta (frecuencia, duración en ms)
    except:
        print("Error reproduciendo sonido")

def update_gui(alert):
    alert_text.insert(tk.END, f"[ALERT] {alert}\n", "alert")
    alert_text.see(tk.END)
    play_alert_sound()

def monitor_snort_alerts(log_file):
    if not os.path.exists(log_file):
        alert_text.insert(tk.END, "Log file not found. Please check the Snort configuration.\n", "error")
        return
    
    with open(log_file, "r") as file:
        file.seek(0, os.SEEK_END)
        
        while True:
            line = file.readline()
            if line:
                root.after(0, update_gui, line.strip())
            else:
                time.sleep(1)

# Configurar la ventana Tkinter
root = tk.Tk()
root.title("Snort IDS Alert Monitor")
root.geometry("600x400")

alert_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=80, height=20)
alert_text.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
alert_text.tag_config("alert", foreground="red")
alert_text.tag_config("error", foreground="blue")

log_path = "/var/log/snort/alerts.log"  # Ajusta la ruta si es necesario

# Ejecutar el monitoreo en un hilo separado
import threading
threading.Thread(target=monitor_snort_alerts, args=(log_path,), daemon=True).start()

root.mainloop()
